## Previsao_Acoes.ipynb (estrutura de Markdown e código explicativo)

## =========================================
## PROJETO: PREVISÃO DE AÇÕES COM LSTM (Keras/TensorFlow)
## =========================================

### Motivação
Preve# Projeto: Previsão de Preços de Ações com Redes LSTM

Este projeto utiliza redes neurais recorrentes (RNN), especificamente LSTM (Long Short-Term Memory), para prever o próximo valor de fechamento ajustado de uma ação da B3 (Bolsa de Valores do Brasil).

**Tecnologias utilizadas:** `Python`, `Pandas`, `Scikit-Learn`, `TensorFlow/Keras`, `yfinance`, `Flask`

In [1]:
#!pip install yfinance pandas numpy scikit-learn tensorflow matplotlib

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout

## 1. Coleta dos Dados
Usamos a biblioteca `yfinance` para coletar dados históricos da ação desejada.

A ação `PETR4.SA` (Petrobras PN) será usada como exemplo.

In [2]:

# Escolha uma ação (use o ticker com ".SA" para ações brasileiras)
ticker = 'PETR4.SA'
dados = yf.download(ticker,
                    start='2018-01-01',
                    end='2024-05-10',
                    auto_adjust=True)

# Vamos usar apenas o preço de fechamento 'Close'
dados_fechamento = dados['Close'].values.reshape(-1, 1)

print(dados.head())

[*********************100%***********************]  1 of 1 completed

Price          Close      High       Low      Open    Volume
Ticker      PETR4.SA  PETR4.SA  PETR4.SA  PETR4.SA  PETR4.SA
Date                                                        
2018-01-02  6.077724  6.077724  5.945520  5.945520  33461800
2018-01-03  6.132811  6.140155  6.011624  6.055692  55940900
2018-01-04  6.143827  6.228291  6.103432  6.162190  37064900
2018-01-05  6.180550  6.191567  6.085069  6.132810  26958200
2018-01-08  6.253998  6.253998  6.136482  6.147500  28400000


In [3]:
print(dados.tail())

Price           Close       High        Low       Open    Volume
Ticker       PETR4.SA   PETR4.SA   PETR4.SA   PETR4.SA  PETR4.SA
Date                                                            
2024-05-03  35.209023  35.915144  34.820656  35.915144  45114200
2024-05-06  35.447342  35.623873  35.058976  35.191372  22977200
2024-05-07  35.879841  35.879841  35.306115  35.438514  23808600
2024-05-08  36.427090  36.480050  35.579744  35.615047  25352000
2024-05-09  36.780144  36.930196  36.135808  36.241728  27206400


## 2. Pré-processamento
Os dados são normalizados entre 0 e 1 para melhor desempenho da rede LSTM. Em seguida, criamos sequências dos últimos 60 dias para prever o próximo valor.

In [4]:
# Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
dados_scaled = scaler.fit_transform(dados_fechamento)

# Criar os dados de treino em sequências
X_train = []
y_train = []

tamanho_sequencia = 60 # Olhar para os últimos 60 dias

for i in range(tamanho_sequencia, len(dados_scaled)):
    X_train.append(dados_scaled[i-tamanho_sequencia:i, 0])
    y_train.append(dados_scaled[i, 0])

# Converter para numpy arrays
import numpy as np
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape para o formato da LSTM [samples, timesteps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

## 3. Construção do Modelo
A arquitetura LSTM usada tem duas camadas LSTM e camadas de Dropout para evitar overfitting.

In [5]:
# PASSO 1: Construa o modelo usando o 'Input' que acabamos de importar
modelo = Sequential([
    # Agora Python sabe o que é 'Input' porque o importamos acima
    Input(shape=(X_train.shape[1], 1)),

    LSTM(units=50, return_sequences=True),
    Dropout(0.2),

    LSTM(units=50, return_sequences=False),
    Dropout(0.2),

    Dense(units=1)
])

# PASSO 2: Compile e visualize (sem alterações aqui)
modelo.compile(optimizer='adam', loss='mean_squared_error')

# Visualizar a arquitetura
modelo.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 60, 50)              │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 60, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,651 (119.73 KB)

 Trainable params: 30,651 (119.73 KB)

 Non-trainable params: 0 (0.00 B)

## 4. Treinamento
Treinamos a rede por 25 épocas usando um batch size de 32.

In [6]:
# Treinar o modelo
historico = modelo.fit(X_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 12s 58ms/step - loss: 0.0448
Epoch 2/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0027
Epoch 3/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 0.0019
Epoch 4/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - loss: 0.0019
Epoch 5/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0020
Epoch 6/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.0019
Epoch 7/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - loss: 0.0017
Epoch 8/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0017
Epoch 9/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0014
Epoch 10/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.0014
Epoch 11/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.0019
Epoch 12/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - loss: 0.0016
Epoch 13/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 0.0015
Epoch 14/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 0.0015
Epoch 15/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - loss: 0.0018
Epo

## 5. Previsão
Usamos os últimos 60 dias da série para prever o preço de fechamento do próximo dia útil.

In [7]:
# Para testar, precisamos dos últimos 60 dias do nosso dataset
ultimos_60_dias = dados_fechamento[-tamanho_sequencia:]

# Normalizar esses dados usando o mesmo scaler
ultimos_60_dias_scaled = scaler.transform(ultimos_60_dias)

# Criar o X_test
X_test = []
X_test.append(ultimos_60_dias_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Fazer a previsão
preco_previsto_scaled = modelo.predict(X_test)

# Reverter a normalização para obter o preço em R$
preco_previsto = scaler.inverse_transform(preco_previsto_scaled)

print(f"Previsão do preço de fechamento para o próximo dia: R$ {preco_previsto[0][0]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 670ms/step
Previsão do preço de fechamento para o próximo dia: R$ 35.61


## Conclusão

- O modelo foi capaz de aprender padrões históricos da série de preços.
- LSTM é adequado para problemas de séries temporais pois mantém dependências de longo prazo.
- O projeto pode ser estendido com múltiplos atributos (volume, abertura, etc.) ou modelos mais complexos (CNN-LSTM, Attention).